# Optimización de Portafolio 28 días activo

Este código utiliza la biblioteca **PyPortfolioOpt** para construir un portafolio de inversión que maximiza el **Ratio de Sharpe** ajustado a un horizonte mensual (28 días hábiles). El proceso incluye los siguientes pasos:

# Read csv

In [27]:
def get_df():

    import pandas as pd  # For data manipulation and analysis
    import numpy as np  # For numerical operations
    import requests  # For making HTTP requests to fetch data from web pages
    import os  # For interacting with the operating system (e.g., file paths)
    import yfinance as yf  # For fetching financial data from Yahoo Finance

    # PyPortfolioOpt library for portfolio optimization
    from pypfopt.efficient_frontier import EfficientFrontier  # For creating efficient frontier and optimizing portfolios
    from pypfopt import risk_models  # For calculating risk models (e.g., covariance matrix)
    from pypfopt import expected_returns  # For calculating expected returns

    # PyPortfolioOpt library for discrete allocation
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices  # For discrete allocation of portfolio weights

    import requests  # Duplicate import, already imported above
    from bs4 import BeautifulSoup  # For parsing HTML and XML documents
    import pandas as pd  # Duplicate import, already imported above

    from datetime import datetime  # For manipulating dates and times

    from forex_python.converter import CurrencyRates
    import pandas as pd
    from pypfopt import expected_returns, risk_models

    # Function to read stock symbols from a file
    def read_stock_symbols(file_path):
        with open(file_path, 'r') as file:
            return file.read().splitlines()

    # Function to fetch adjusted close prices for given symbols
    def fetch_adj_close_prices(symbols):
        adj_close_list = []
        for symbol in symbols:
            stock = yf.Ticker(symbol)
            hist = stock.history(period="max")

            # Reset index to work with the datetime as a column
            hist.reset_index(inplace=True)

            # Extract only the date, ignoring the hour
            hist['Date'] = hist['Date'].dt.date

            # Group by date and take the last closing price of each day
            daily_data = hist.groupby('Date').agg({'Close': 'last'}).rename(columns={'Close': symbol})
            
            # Append the data to the list
            adj_close_list.append(daily_data)

        # Concatenate all data along the columns axis
        return pd.concat(adj_close_list, axis=1)


    # Function to generate all possible trading dates
    def generate_all_dates(start_date, end_date):
        return pd.date_range(start=start_date, end=end_date, freq='B').date

    current_date = datetime.now().strftime("%Y-%m-%d")

    csv_file = f"{current_date}.csv"

    if not os.path.exists(csv_file):
        # Read stock symbols from file
        stock_symbols = read_stock_symbols('ETFS&Stocks.txt')
        stock_symbols.insert(0, '^GSPC')  # Add S&P 500 index symbol

        # Fetch data for all stock symbols
        df = fetch_adj_close_prices(stock_symbols)

        # Get the minimum and maximum dates from the fetched data
        min_date = df.index.min()
        max_date = df.index.max()

        # Generate a complete list of business dates
        all_dates = pd.DataFrame(generate_all_dates(min_date, max_date), columns=['Date'])

        # Merge the generated dates with the fetched data, filling missing values
        df.reset_index(inplace=True)  # Reset index to make 'Date' a column
        merged_df = all_dates.merge(df, on='Date', how='left')

        # Filter out the dates where the S&P 500 index (^GSPC) has missing values
        # merged_df = merged_df[merged_df['^GSPC'].notna()]

        # Set 'Date' as index again for saving to CSV
        merged_df.set_index('Date', inplace=True)

        # Save the cleaned data to a CSV
        merged_df.to_csv(csv_file, index=True)

    df = pd.read_csv(csv_file, index_col='Date', parse_dates=True)

    # Ensure the "Results" folder exists
    if not os.path.exists("Portafolios"):
        os.makedirs("Portafolios")

    # No se porque con estas da error
    columns_to_drop = [
        "AAXJ", "ACWI", "BIL", "BOTZ", "DIA", "EEM", "EWZ", "FAS", "GDX", "GLD",
        "IAU", "ICLN", "INDA", "IVV", "KWEB", "LIT", "MCHI", "NAFTRACISHRS.MX", "PSQ", "QCLN"
    ]

    # Drop the columns
    df.drop(columns=columns_to_drop, inplace=True)

    return df



In [28]:
df = get_df()

# Only etfs or etfs&stocks

ETF's & Stocks

Only Stocks

In [39]:
def remove_etfs(df):    
# Read the column names from ETFS.txt
    # Additional columns to drop
    additional_columns_to_drop = [
        "SPY", "SQQQ", "TAN", "TECL", "TLT", "TNA", "TQQQ", "USO", "VEA", "VGT",
        "VNQ", "VOO", "VT", "VTI", "VWO", "VYM", "XLE", "XLF", "XLK", "XLV"
    ]

    # Drop the columns
    df.drop(columns=additional_columns_to_drop, inplace=True, errors='ignore')

    return df

In [40]:
remove_etfs(df)

,^GSPC,AA1.MX,AAL,AAPL,ABBV,ABNB,AC,ACTINVRB.MX,AFRM,AGNC,...,VESTA.MX,VOLARA.MX,VZ,WALMEX.MX,WFC,WMT,WYNN,X,XOM,ZM
Date,,,,,,,,,,,,,,,,,,,,,
1927-12-30,17.660000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-03,17.760000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-04,17.719999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1928-01-05,17.549999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-30,5762.479980,696.01001,11.24,233.000000,197.479996,126.809998,35.419998,16.309999,40.820000,10.46,...,53.070000,12.40,44.910000,59.410000,56.490002,80.750000,95.879997,35.330002,117.220001,69.739998
2024-10-01,5708.750000,NaN,10.90,226.210007,196.960007,125.470001,35.410000,NaN,38.480000,10.27,...,NaN,NaN,45.209999,NaN,55.389999,81.269997,99.449997,35.000000,119.930000,68.639999
2024-10-02,5709.540039,696.01001,10.74,226.779999,196.820007,126.180000,35.599998,16.320000,39.049999,10.30,...,51.439999,12.37,45.049999,57.610001,55.299999,80.430000,102.040001,34.599998,121.519997,67.610001


In [50]:
def portapara(df, portafolio, days, rf):

    import pandas as pd  # For data manipulation and analysis
    import numpy as np  # For numerical operations
    import requests  # For making HTTP requests to fetch data from web pages
    import os  # For interacting with the operating system (e.g., file paths)
    import yfinance as yf  # For fetching financial data from Yahoo Finance

    # PyPortfolioOpt library for portfolio optimization
    from pypfopt.efficient_frontier import EfficientFrontier  # For creating efficient frontier and optimizing portfolios
    from pypfopt import risk_models  # For calculating risk models (e.g., covariance matrix)
    from pypfopt import expected_returns  # For calculating expected returns

    # PyPortfolioOpt library for discrete allocation
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices  # For discrete allocation of portfolio weights

    from datetime import datetime  # For manipulating dates and times
    from openpyxl import load_workbook  # For saving to Excel

    mxntodlls = .05
    portafolio = portafolio * mxntodlls

    rf = (1 + rf) ** (252 / days) - 1
    
    mu = expected_returns.mean_historical_return(df, frequency=days)
    s = risk_models.sample_cov(df, frequency=days)

    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

    def save_to_excel(file_path, expected_return, volatility, sharpe_ratio, rf, leftover):
        workbook = load_workbook(file_path)
        sheet = workbook.active

        sheet["E2"] = "Return"
        sheet["E3"] = "Volatility"
        sheet["E4"] = "Sharpe Ratio"
        sheet["E5"] = "Risk-Free Rate"
        sheet["E6"] = "Leftover"
        sheet["F2"] = expected_return
        sheet["F3"] = volatility
        sheet["F4"] = sharpe_ratio
        sheet["F5"] = rf
        sheet["F6"] = leftover

        workbook.save(file_path)

    class PortfolioOptimization:
        def __init__(self, mu, s, portafolio, rf):
            self.mu = mu
            self.s = s
            self.portafolio = portafolio
            self.rf = rf

        def max_sharpe(self):
            ef = EfficientFrontier(self.mu, self.s)
            weights = ef.max_sharpe(risk_free_rate=self.rf)
            cleaned_weights = ef.clean_weights()

            performance = ef.portfolio_performance(verbose=True, risk_free_rate=self.rf)
            expected_return, volatility, sharpe_ratio = performance

            latest_prices = get_latest_prices(df)
            da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=self.portafolio)
            allocation, leftover = da.lp_portfolio()

            discrete_allocation_list = [allocation[symbol] for symbol in allocation]

            portfolio_df = pd.DataFrame({
                'Company Ticker': allocation.keys(),
                'Discrete Allocation': discrete_allocation_list
            })

            file_path = f"Portafolios/Max Sharpe {current_date}.xlsx"
            portfolio_df.to_excel(file_path, index=False)
            save_to_excel(file_path, expected_return, volatility, sharpe_ratio, self.rf, leftover)

        def max_return(self):
            ef = EfficientFrontier(self.mu, self.s)
            weights = ef.max_quadratic_utility()
            cleaned_weights = ef.clean_weights()

            performance = ef.portfolio_performance(verbose=True, risk_free_rate=self.rf)
            expected_return, volatility, sharpe_ratio = performance

            latest_prices = get_latest_prices(df)
            da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=self.portafolio)
            allocation, leftover = da.lp_portfolio()

            discrete_allocation_list = [allocation[symbol] for symbol in allocation]

            portfolio_df = pd.DataFrame({
                'Company Ticker': allocation.keys(),
                'Discrete Allocation': discrete_allocation_list
            })

            file_path = f"Portafolios/Max Return {current_date}.xlsx"
            portfolio_df.to_excel(file_path, index=False)
            save_to_excel(file_path, expected_return, volatility, sharpe_ratio, self.rf, leftover)

    return PortfolioOptimization(mu, s, portafolio, rf)


In [46]:
portafolio = 1000000
days = 252
rf = 0.0001

In [51]:
optimizer = portapara(df, portafolio, days, rf)
optimizer.max_return()


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.

Expected annual return: 38.6%
Annual volatility: 34.1%
Sharpe Ratio: 1.13


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


In [19]:
def portapara(df, portafolio, days, rf):

    import pandas as pd  # For data manipulation and analysis
    import numpy as np  # For numerical operations
    from pypfopt.efficient_frontier import EfficientFrontier  # For optimizing portfolios
    from pypfopt import risk_models  # For calculating risk models (e.g., covariance matrix)
    from pypfopt import expected_returns  # For calculating expected returns
    from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices  # For discrete allocation of portfolio weights
    from datetime import datetime  # For manipulating dates and times
    from openpyxl import load_workbook  # For saving to Excel

    mxntodlls = .05
    portafolio = portafolio * mxntodlls

    rf = (1 + rf) ** (252 / days) - 1
    mu = expected_returns.mean_historical_return(df, frequency=days)
    s = risk_models.sample_cov(df, frequency=days)

    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

    def save_to_excel(file_path, expected_return, volatility, sharpe_ratio, rf, leftover):
        workbook = load_workbook(file_path)
        sheet = workbook.active

        sheet["E2"] = "Return"
        sheet["E3"] = "Volatility"
        sheet["E4"] = "Sharpe Ratio"
        sheet["E5"] = "Risk-Free Rate"
        sheet["E6"] = "Leftover"
        sheet["F2"] = expected_return
        sheet["F3"] = volatility
        sheet["F4"] = sharpe_ratio
        sheet["F5"] = rf
        sheet["F6"] = leftover

        workbook.save(file_path)

    class PortfolioOptimizer:
        def __init__(self):
            self.mu = mu
            self.s = s
            self.portafolio = portafolio
            self.rf = rf

        def max_sharpe(self):
            ef = EfficientFrontier(self.mu, self.s)
            weights = ef.max_sharpe(risk_free_rate=self.rf)
            cleaned_weights = ef.clean_weights()
            performance = ef.portfolio_performance(verbose=True, risk_free_rate=self.rf)
            expected_return, volatility, sharpe_ratio = performance

            latest_prices = get_latest_prices(df)
            da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=self.portafolio)
            allocation, leftover = da.lp_portfolio()

            discrete_allocation_list = [allocation[symbol] for symbol in allocation]

            portfolio_df = pd.DataFrame({
                'Company Ticker': allocation.keys(),
                'Discrete Allocation': discrete_allocation_list
            })

            file_path = f"Portafolios/Max Sharpe {current_date}.xlsx"
            portfolio_df.to_excel(file_path, index=False)
            save_to_excel(file_path, expected_return, volatility, sharpe_ratio, self.rf, leftover)

    return PortfolioOptimizer()



In [23]:
optimizer = portapara(df, 1000000, 1, 0.00000001)
optimizer.max_sharpe()


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\expected_returns.py:56: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna(how="all")
c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.

Expected annual return: 0.1%
Annual volatility: 0.7%
Sharpe Ratio: 0.09


c:\Users\herie\AppData\Local\Programs\Python\Python312\Lib\site-packages\cvxpy\problems\problem.py:1407: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


# Import Company Name

In [16]:
from openpyxl import load_workbook

def save_to_excel(file_path, expected_return, volatility, sharpe_ratio):
    
    # Load the workbook and select the active sheet
    workbook = load_workbook(file_path)
    sheet = workbook.active

    # Write the performance metrics to the specified cells
    sheet["E2"] = "Return"
    sheet["E3"] = "Volatility"
    sheet["E4"] = "Sharpe Ratio"
    sheet["F2"] = expected_return
    sheet["F3"] = volatility
    sheet["F4"] = sharpe_ratio

    # Save the workbook
    workbook.save(file_path)

# Max Sharpe Ratio

In [45]:
# Function to optimize portfolio for maximum Sharpe ratio
def optimize_for_max_sharpe(df, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for maximum Sharpe ratio
    weights = ef.max_sharpe(risk_free_rate=rf)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Max Sharpe {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [ ]:
portfolio_max_sharpe = optimize_for_max_sharpe(df, portfolio_value=portafolio)
print(portfolio_max_sharpe)

# Target Volatility

In [232]:
# Function to optimize portfolio for target volatility
def optimize_for_target_volatility(df, target_volatility, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for target volatility
    weights = ef.efficient_risk(target_volatility)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Target Volatility {target_volatility} {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [ ]:
target_volatility = 0.15

portafolio_target_volatility = optimize_for_target_volatility(df, target_volatility, portafolio)
print(portafolio_target_volatility)

# Target Return

In [234]:
# Function to optimize portfolio for target return
def optimize_for_target_return(df, target_return, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for target return
    weights = ef.efficient_return(target_return)
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M-%S")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Target Return {target_return} {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df

In [ ]:
portafolio_target_return = optimize_for_target_return(df, 0.002, portafolio)
print(portafolio_target_return)

In [ ]:
portafolio_target_return = optimize_for_target_return(df, 0.001, portafolio)
print(portafolio_target_return)

In [ ]:
portafolio_target_return = optimize_for_target_return(df, 0.003, portafolio)
print(portafolio_target_return)

# Max Return

In [20]:
# Function to optimize portfolio for maximum return
def optimize_for_max_return(df, portfolio_value):

    ef = EfficientFrontier(mu, s)

    # Optimize for maximum return
    weights = ef.max_quadratic_utility()
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

     # Get the current date
    current_date = datetime.now().strftime("%Y-%m-%d-%H-%M")

    file_path = f"Portafolios/Max return {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    # Return the DataFrame
    return portfolio_df, leftover

In [ ]:
portafolio_max_return, leftover = optimize_for_max_return(df, portfolio_value=portafolio)
print(portafolio_max_return, leftover)

# Min variance

In [240]:
# Function to optimize portfolio for minimum variance
def optimize_for_min_variance(df, portfolio_value):
    ef = EfficientFrontier(mu, s)

    # Optimize for minimum variance
    weights = ef.min_volatility()
    cleaned_weights = ef.clean_weights()

    # Get portfolio performance
    performance = ef.portfolio_performance(verbose=True, risk_free_rate=rf)
    expected_return, volatility, sharpe_ratio = performance

    # Get latest prices and perform discrete allocation
    latest_prices = get_latest_prices(df)
    da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=portfolio_value)
    allocation, leftover = da.lp_portfolio()

    # Get company names for the allocation
    discrete_allocation_list = [allocation[symbol] for symbol in allocation]

    # Create a DataFrame for the portfolio
    portfolio_df = pd.DataFrame({
        'Company Ticker': allocation.keys(),
        'Discrete Allocation': discrete_allocation_list
    })

    # Get the current date
    current_date = datetime.now().strftime("%d-%m-%Y-%H-%M")

    # Save the portfolio allocation to an Excel file in the "Results" folder with the current date in the filename
    file_path = f"Portafolios/Min variance {current_date}.xlsx"
    portfolio_df.to_excel(file_path, index=False)

    # Write the performance metrics to the Excel file
    save_to_excel(file_path, expected_return, volatility, sharpe_ratio)
    
    return portfolio_df

In [ ]:
portafolio_min_variance = optimize_for_min_variance(df, portfolio_value=portafolio)
print(portafolio_min_variance)